In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''
  Importing required libraries
'''
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import os
#---------------------------------
def check(img,a,b,k,sig):
    a = (a+1)//2 if a%2==1 else a//2
    b = (b+1)//2 if b%2==1 else b//2
    img = cv2.resize(img,(b,a))
    nxt = cv2.GaussianBlur(img,(k,k),sig)
    return nxt,a,b
#----------------------------------
'''
    Gaussian_Pyramid
    img = image, n = number of levels
    first level is considered as level 0
'''
def Gaussian_Pyramid(img,n,k=5,sig=1):
    pyramids = {}
    a,b = img.shape
    for level in range(n):
        pyramids[level] = cv2.GaussianBlur(img,(k,k),sig)
        a = a//2; b = b//2
        img = cv2.resize(pyramids[level],(b,a))
    return pyramids
'''
    Laplacian_Pyramid
    img = image, n = number of levels
    first level is considered as level 0
    The following function returns a dictionary containing laplacian pyramids
'''
def Laplacian_Pyramid(img,n,k=5,sig = 1):
    pyramids = {}           
    a,b = img.shape    
    prev = cv2.GaussianBlur(img,(k,k),sig)
    for level in range(n):
        f1 = b; f2 = a
        nxt,a,b = check(prev,a,b,k,sig)
        lap = cv2.subtract(prev , cv2.resize(nxt,(f1,f2)))
        pyramids[level] = lap
        prev = nxt
    return pyramids

#-----------------------------
'''
code for the gradient of I
'''

def gradient_I(img):
    Ix = cv2.Sobel(img,-1,0,1)
    Iy = cv2.Sobel(img,-1,1,0)
    return Ix,Iy
#------------------------------
'''
code for separating frames from videos and storing it in a destination location 
'''
def sep_frames_vid(video_path,dest_loc):
    cap= cv2.VideoCapture(video_path)
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        cv2.imwrite(dest_loc+'/'+'%06d.jpg'%i,frame)
        i+=1
 
    cap.release()
    cv2.destroyAllWindows()

#-----------------------------------------------------
'''
img difference between 2 images
'''

def img_difference(img1,img2):
    diff = img1-img2
    diff = abs(diff)
    diff = np.int32(diff)
    return diff
#-----------------------------------------------------
'''
These 2 codes find the (u,v) estimates for the total image
based on the value of a-vector
'''
def U(x,y,a):
    X = np.array([[1,x,y,0,0,0],[0,0,0,1,x,y]])
    if a.shape== (6,1):
        U_ = np.matmul(X,a)
    else:
        a = a.reshape((6,1))
        U_ = np.matmul(X,a)
    return U_
'''
  This function calculates U(x) of
  entire image from a-vector
'''
def U_img(x_dim,y_dim,a):
    U_im = np.zeros((x_dim,y_dim,2))
    for i in range(x_dim):
        for j in range(y_dim):
            u_tmp = U(i,j,a)
            U_im[i,j,0] = u_tmp[0,0]
            U_im[i,j,1] = u_tmp[1,0]
    return U_im

#------------------------------------------------------
#code for image modification
'''
inputs  ref_img --- reference image I(t)
        img_unmod --- unmodified image I(t-1)
        U_img ---- (u,v) values matrix 
'''

def img_modification(ref_img,img_unmod,U_img):
    img_mod = np.zeros(img_unmod.shape)             #initializing the image modified
    x_max,y_max = img_unmod.shape
    for i in range(x_max):                          #for every (i,j) in the unmodified image
        for j in range(y_max):
            u1,v1 = np.int32(np.floor(U_img[i,j,:]))    #flooring (u,v)
            u2,v2 = np.int32(np.ceil(U_img[i,j,:]))     #ceiling (u,v)
            if (np.array([i-u1,i-u2])<x_max).all() and (np.array([i-u1,i-u2])>=0).all() and (np.array([j-v1,j-v2])<y_max).all() and (np.array([j-v1,j-v2])>=0).all():
                if  abs(ref_img[i,j]-img_unmod[i-u1,j-v1])<abs(ref_img[i,j]-img_unmod[i-u2,j-v2]): #comparing how close is the pixel intensity to that of the reference image at the same location
                    img_mod[i,j] = img_unmod[i-u1,j-v1]
                else:
                    img_mod[i,j] = img_unmod[i-u2,j-v2]
    return img_mod
#-----------------------------------------------------
'''
img1 = I(t-1)                                                                   #look at page 5 starting paragraph
img2 = I(t) ------ this is my reference image.
Look at eq (7) in the paper which is of the form L*(delta(a)) = R

'''
#Ensure both the images are of the same size
def affine_model(img1,img2,a,iter):
    Ix,Iy = gradient_I(img2)                                  #gradient of the reference image
    m_max,n_max = img1.shape                                  #Shape of the image
    U_prev = np.zeros((m_max,n_max,2))                        # Initializing U(x)
    img1_mod = img1.copy()                                    
    for it in range(iter):
        L = 0                                                 #LHS of the equation(present in the paper)
        R = 0                                                 #RHS
        delta_I = img2 - img1_mod                             #delta_I calculation
        for i in range(m_max):
            for j in range(n_max):
                X = np.array([[1,i,j,0,0,0],[0,0,0,1,i,j]])
                grad_tmp = np.array([[Ix[i,j]],[Iy[i,j]]])    #gradient at position (i,j)
                L1 = np.matmul(X.T,grad_tmp)                  
                L2 = np.matmul(L1,L1.T)                       #L calculation at (i,j)
                L += L2                                       #L calculation
                R1 = L1*delta_I[i,j]                          #R calculation at (i,j)
                R +=R1                                        #R calculation
        R = -1*R
        delta_a = np.matmul(np.linalg.inv(L),R)               #delta_a calculation
        a+=delta_a                                            #Updating delta_a
        U_prev = U_img(m_max,n_max,a)                         #Updating U_prev
        img1_mod = img_modification(img2,img1,U_prev)         #Modifying img1 using U_prev
    return a ,img1_mod
#-------------------------------------------------------------------------------------
'''
  Hierarchical affine model
  level 0 is considered as first level
  Here k represents size of the gaussian kernal (k,k) and sigma variation
'''
def hier_affine(img1,img2,no_of_levels=3,pyramid = 'Gaussian' ,iter_per_level = 4, k=3,sig = 1): 
    if pyramid == 'Gaussian':                                 
        py_img1 = Gaussian_Pyramid(img1,no_of_levels,k,sig)     #Gaussian Pyramid of img1
        py_img2 = Gaussian_Pyramid(img2,no_of_levels,k,sig)     #Gaussian Pyramid of img2
    else:
        py_img1 = Laplacian_Pyramid(img1,no_of_levels,k,sig)    #Laplacian Pyramid of img1
        py_img2 = Laplacian_Pyramid(img2,no_of_levels,k,sig)    #Laplacian Pyramid of img2
    a = np.zeros((6,1))                                         #Initializing parameters (a)
    for i in range(no_of_levels-1,-1,-1):
        a ,img1_mod = affine_model(py_img1[i],py_img2[i],a,iter = iter_per_level)#Updates parameter each level
        print(i)                                                #Prints level number
    return a,img1_mod


RUN THE CODE FROM HERE

In [ ]:
img1 = cv2.imread('/content/drive/Shared drives/CV_Project/Dataset/affine model/clouds_frames/000082.jpg')  #input images
img2 = cv2.imread('/content/drive/Shared drives/CV_Project/Dataset/affine model/clouds_frames/000083.jpg')
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)                                        #converting the images to grayscale
img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
a,img_mod = hier_affine(img1,img2,no_of_levels = 3,pyramid = 'Laplacian')

In [ ]:
#calculating the output image
x_dim,y_dim = img1.shape
U_= U_img(x_dim,y_dim,a)
out = img_modification(img2,img1,U_)
plt.imshow(out,cmap='gray')

In [ ]:
#raw difference
diff = img_difference(img1,img2)
plt.figure(figsize=(8,11))
plt.imshow(diff,cmap='gray')

In [ ]:
#compensated difference
diff = img_difference(out,img2)
plt.figure(figsize=(8,11))
plt.imshow(diff,cmap='gray')